In [ ]:
import os
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import cPickle as pickle

# pfam version
PFAM = 'pfam-v31'

# Getting path
curr_dir = os.getcwd()
files_dir = curr_dir+'/../' 
spider_dir = curr_dir+'/SPIDER2/protein_seq_results/'
    
# Diff between v30 and v31
with open(spider_dir+"/../../v30-v31_diff_domains_genes_prot_seq.pik", 'rb') as handle:
    diff_genes = pickle.load(handle)
    
# The dictionary of genes -> proteins -> sequence that will be run
sequence_dict = diff_genes

In [ ]:
dirfiles = !ls -t $'./SPIDER2/protein_seq_results/out'

# Only look at new out jobs
threshold = 4018018

for f in dirfiles:
    # Only check out files
    if f[len(f)-3:len(f)] != 'out' or int(f.split("-")[1].split(".")[0]) <= threshold:
        continue
    # Check for print statement at end
    out = open(spider_dir+'/out/'+f).read()
    if not 'SUCCESS!!!' in out or 'Error' in out:
        print(f)

In [ ]:
# Rerunning might take a bit, so only do it if you're sure
import subprocess
rerun_list = [] # Add items in the form <gene_id>-<prot_id> e.g. ENSG00000085276.13-ENSP00000417899.1

for term in rerun_list:
    gene = term.split("-")[0]
    prot = term.split("-")[1]
    # Save sequence to file
    filename = files_dir+"/SPIDER/seq/"+term.replace('.','-')+'.seq'
    with open(filename,'w') as f:
        f.write(sequence_dict[gene][prot].replace('*',''))
    f.close()

    # Run SPIDER2
    subprocess.call([files_dir+'/SPIDER/misc/run_local.sh',filename])